Importing tools and dataset

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd
import seaborn as sns

In [3]:
data = pd.read_csv("C:/Users/Admin/Desktop/Last Sem Degree/PSM2/Coding/data.csv")
data.head(10)

# Display the first few rows of the dataframe to inspect the data
print("Original Data:")
print(data.head())

# Display data types to identify any non-numeric columns
print("\nData Types:")
print(data.dtypes)

# Check for any non-numeric values in the data
print("\nNon-numeric Values:")
print(data[~data.map(np.isreal)])


Original Data:
   Unnamed: 0   X1   X2   X3   X4   X5   X6   X7   X8   X9  ...  X170  X171  \
0  X21.V1.791  135  190  229  223  192  125   55   -9  -33  ...   -17   -15   
1  X15.V1.924  386  382  356  331  320  315  307  272  244  ...   164   150   
2     X8.V1.1  -32  -39  -47  -37  -32  -36  -57  -73  -85  ...    57    64   
3   X16.V1.60 -105 -101  -96  -92  -89  -95 -102 -100  -87  ...   -82   -81   
4   X20.V1.54   -9  -65  -98 -102  -78  -48  -16    0  -21  ...     4     2   

   X172  X173  X174  X175  X176  X177  X178  y  
0   -31   -77  -103  -127  -116   -83   -51  4  
1   146   152   157   156   154   143   129  1  
2    48    19   -12   -30   -35   -35   -36  5  
3   -80   -77   -85   -77   -72   -69   -65  5  
4   -12   -32   -41   -65   -83   -89   -73  5  

[5 rows x 180 columns]

Data Types:
Unnamed: 0    object
X1             int64
X2             int64
X3             int64
X4             int64
               ...  
X175           int64
X176           int64
X177       

In [5]:
# Display the data types of each column
print("Data Types of Each Column:")
print(data.dtypes)

# Count the number of columns by data type
print("\nNumber of Columns by Data Type:")
print(data.dtypes.value_counts())


Data Types of Each Column:
Unnamed: 0    object
X1             int64
X2             int64
X3             int64
X4             int64
               ...  
X175           int64
X176           int64
X177           int64
X178           int64
y              int64
Length: 180, dtype: object

Number of Columns by Data Type:
int64     179
object      1
Name: count, dtype: int64


In [4]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())


Missing Values:
Unnamed: 0    0
X1            0
X2            0
X3            0
X4            0
             ..
X175          0
X176          0
X177          0
X178          0
y             0
Length: 180, dtype: int64


In [6]:
# check for null values
print("Null values found:")
print(data.isnull().sum()) # sum null values in columns

# Check for negative values
print("\nNegative values found:")
numerical_columns = data.select_dtypes(include=['int64','float64']). columns # select numerical columns
negative_values = (data[numerical_columns]< 0 ).sum() # Count negative value per column
print(negative_values)

Null values found:
Unnamed: 0    0
X1            0
X2            0
X3            0
X4            0
             ..
X175          0
X176          0
X177          0
X178          0
y             0
Length: 180, dtype: int64

Negative values found:
X1      6348
X2      6328
X3      6265
X4      6272
X5      6337
        ... 
X175    6412
X176    6423
X177    6416
X178    6387
y          0
Length: 179, dtype: int64


In [7]:
# check for non numerical columns
non_numeric_columns = data.select_dtypes(exclude=['int64','float64']).columns           # filter column that are not numerical (e.g. strings, dates, etc)

# Print Result
if len(non_numeric_columns) > 0:
    print("\nNon-numeric columns found:")
    print(non_numeric_columns)
    print(f"\nNumber of non-numeric columns: {len(non_numeric_columns)}")                  # Show how many non-numeric columns
else:
    print("All column in the dataset are numerical.")


Non-numeric columns found:
Index(['Unnamed: 0'], dtype='object')

Number of non-numeric columns: 1
